In [1]:
import requests 
import pandas as pd
import matplotlib.pyplot as plt
import datetime 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,StandardScaler,OneHotEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout,Flatten,Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Concatenate


%matplotlib inline

In [1]:
'''tansformations with OOP'''

class DataTransformer:
    def __init__(self,df):
        self.df = df
    '''Class containing methods to transform the imported data'''

    # in OOP do u not need to call return?
    def transform(self):
        '''overall transformation of data'''
        self.interpolate()
        self.add_cyclical_features()
        self.add_time_features()
        self.ohe()
        self.add_historical_windpower()
        self.add_momentum_force()
        self.scale()
        return self.df

    def interpolate(self):
        '''interpolation of data'''
        df = self.df
        df['Time'] = df['Time'].apply(lambda x : datetime.datetime.strptime(x[:-3], '%Y/%m/%d %H:%M'))
        df['Time'] = pd.to_datetime(df['Time'])    #why double the time conversion?
        df.set_index('Time',inplace=True)  
        df = df.resample('1H').asfreq()    #unsure about resample and asfreq
        df.interpolate(method='cubic',axis=0,limit_direction='both',inplace=True)
        self.df = df

    def add_cyclical_features(self):
        '''converts direction into cylical inputs'''
        df = self.df
        cols = df.columns 
        for c in cols:
            if 'Direction' in c:
                df[c+'_norm'] = df[c]/360
                df[c+'_sin'] = df[c+'_norm'].apply(lambda x: np.sin(x))
                df[c+'_cos'] = df[c+'_norm'].apply(lambda x: np.cos(x))
                df.drop([c,c+'_norm'],inplace=True,axis=1)

        self.df = df 

    def scale(self):
        '''normalize entire dataframe'''
        df = self.df
        df = pd.DataFrame(StandardScaler().fit_transform(df),index=df.index,columns=df.columns)
        self.df = df

    def add_time_features(self):
        '''create time inputs as attributes?'''
        df = self.df
        df.reset_index(inplace=True,drop=False)
        #this is assigment of attribute?
        df['hour'] = df['Time'].apply(lambda x: x.hour).astype(str)
        df['month'] = df['Time'].apply(lambda x: x.month).astype(str)
        # df['day'] = df['Time'].apply(lambda x: x.day).astype(str)
        df.set_index('Time',inplace=True)
        self.df = df

    def ohe(self):
        '''One hot encoding of time data'''
        #what is this? I assume it standings for one hot encoding
        #doesn't it affect the entire frame vs just the select month or year?
        df = self.df
        df = pd.get_dummies(df)
        self.df = df

    def add_historical_windpower(self):
        '''conversion of windspeed into windpower'''
        df = self.df
        t = pd.read_csv('target.csv')
        t['Time'] = pd.to_datetime(t['Time'])
        t.set_index('Time',inplace=True)
        #how does this standardscaler object behave?
        target_scaler = StandardScaler().fit(t)
        t = pd.DataFrame(target_scaler.transform(t),index=t.index,columns=t.columns)
        df = df.join(t,how='left')
        self.target_scaler = target_scaler
        self.df = df

    def add_momentum_force(self):
        '''add momentum'''
        time_lag = 18
        df = self.df 
        df['Wind Energy Lag {}'.format(time_lag)] = df['Wind Energy'].shift(time_lag)
        df['Wind Energy Lag {}'.format(2*time_lag)] = df['Wind Energy'].shift(2*time_lag)
        df.dropna(axis=0,inplace=True) ####DROPPING 10 ROWS OF DATA HERE
        # are you not subtracting the future values from present here?
        df['Momentum'] = df['Wind Energy'] - df['Wind Energy Lag {}'.format(time_lag)]
        df['Force'] = df['Wind Energy'] - 2*df['Wind Energy Lag {}'.format(time_lag)] + df['Wind Energy Lag {}'.format(2*time_lag)]
        df.drop(['Wind Energy Lag {}'.format(time_lag),'Wind Energy Lag {}'.format(2*time_lag)],axis=1,inplace=True)
        self.df = df

        ### generate lagged input
        lagged = pd.DataFrame(df['Wind Energy'].shift(1))
        lagged.fillna(method='bfill',inplace=True)
        lagged = StandardScaler().fit_transform(lagged.values)
        self.lagged_input = lagged
        

    #----GETTER Functions---
    #what are they for?

    def get_df(self):
        return self.df

    def get_lagged_input(self):
        return self.lagged_input

    def get_target_scaler(self):
        return self.target_scaler


In [3]:
def add_b_suffix(og_name):
    return_list = []
    for f in og_name:
        return_list.append(f+"-b")
    return return_list

locations = ['guitrancourt', 'lieusaint', 
             'lvs-pussay','parc-du-gatinais', 
             'arville','boissy-la-riviere',
             'angerville-1','angerville-2']

wind_energy = 'energy-ile-de-france'
forecast_endpt = 'https://ai4impact.org/P003/'
analysis_endpt = 'https://ai4impact.org/P003/historical/'

In [4]:
model_1 = locations
model_2 = add_b_suffix(model_1)
models = [model_1, model_2]

In [5]:
model_num = 0
for m in models:
    model_num += 1
    df = pd.read_csv(analysis_endpt+m[0]+'.csv',skiprows=3)
    df.columns = ['Time','Speed_'+m[0],'Direction_'+m[0]]
    df.set_index('Time',inplace=True)
    for i in range(1,len(m)):
        loc = m[i]
        temp = pd.read_csv(analysis_endpt+loc+'.csv',skiprows=3)
        temp.columns = ['Time','Speed_'+loc,'Direction_'+loc]
        temp.set_index('Time',inplace=True)
        df = df.merge(temp,how='left',on='Time')
        df.drop_duplicates(inplace=True)

    df.reset_index(inplace=True,drop=False)
    df.to_csv(f'model_{model_num}.csv')

In [6]:
df1 = pd.read_csv('model_1.csv')
df2 = pd.read_csv('model_2.csv')

In [7]:
target = pd.read_csv('https://ai4impact.org/P003/historical/energy-ile-de-france.csv',header=None)
target.columns = ['Time','Wind Energy']
target.to_csv('target.csv',index=False)
target['Time'] = pd.to_datetime(target['Time'])
target.set_index('Time',inplace=True)

In [2]:
df = df1.copy()
transformer = DataTransformer(df)
transformer.transform()
df = transformer.get_df()
lagged = transformer.get_lagged_input()
df.tail()

NameError: name 'df1' is not defined

In [9]:
X = df.loc[:,df.columns!='Wind Energy'].values
y = df['Wind Energy'].values
X = np.concatenate((X,lagged),axis=1)
X.shape,y.shape

((31765, 64), (31765,))

In [10]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.4)
x_train_features = x_train[:,:62]
x_test_features = x_test[:,:62]
x_train_lagged = x_train[:,62]
x_test_lagged = x_test[:,62]
x_train_features.shape,x_test_features.shape,x_train_lagged.shape,x_test_lagged.shape

((19059, 62), (12706, 62), (19059,), (12706,))

In [11]:
latent_dim = 16
features = Input(shape = (62,), name = 'Features')
def create_encoder(features, latent_dim = latent_dim):
    features = Input(shape = (62,), name = 'Features')
    X = Dense(62, activation = 'relu')(features)
    X = Dense(32, activation = 'relu')(X)
    LATENT = Dense(latent_dim, activation = 'relu')(X)
    encoder = Model(features, LATENT, name = 'encoder')
    return encoder

In [12]:
def create_decoder(latent_dim = latent_dim):
    LATENT_INPUTS = Input(shape = (latent_dim,))
    X = Dense(32, activation = 'relu')(LATENT_INPUTS)
    OUTPUTS = Dense(62, activation = 'linear')(X)
    decoder = Model(LATENT_INPUTS, OUTPUTS, name = 'decoder')
    return decoder

In [13]:
#Difference Model
def energy(latent_dim = latent_dim):
    LATENT_INPUTS = Input(shape = (latent_dim,))
    X = Dense(64,activation='relu')(LATENT_INPUTS)
    X = Dropout(0.2)(X)
    X = Dense(32,activation='relu')(X)
    X = Dense(6,activation='relu')(X)

    output = Dense(1,activation='linear')(X)
    model = Model(inputs= LATENT_INPUTS ,outputs=output, name = 'energy')
    return model

In [14]:
features

<tf.Tensor 'Features:0' shape=(None, 62) dtype=float32>

In [15]:
encoder = create_encoder(62)
decoder = create_decoder()
energy = energy()

In [16]:
autoencoder = Model(features, 
                    decoder(encoder(features)))

In [17]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Features (InputLayer)        [(None, 62)]              0         
_________________________________________________________________
encoder (Model)              (None, 16)                6450      
_________________________________________________________________
decoder (Model)              (None, 62)                2590      
Total params: 9,040
Trainable params: 9,040
Non-trainable params: 0
_________________________________________________________________


In [18]:
def full_model(features):
    full_model = Model(inputs = features, outputs = [decoder(encoder(features)), energy(encoder(features))],
                       name = 'complete')
    full_model.compile(loss = {'energy': 'mse',
                              'decoder': 'mse'},
                      optimizer = 'adam')
    return full_model

In [19]:
full_model = full_model(features)

In [20]:
full_model.summary()

Model: "complete"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Features (InputLayer)           [(None, 62)]         0                                            
__________________________________________________________________________________________________
encoder (Model)                 (None, 16)           6450        Features[0][0]                   
                                                                 Features[0][0]                   
__________________________________________________________________________________________________
decoder (Model)                 (None, 62)           2590        encoder[2][0]                    
__________________________________________________________________________________________________
energy (Model)                  (None, 1)            3373        encoder[3][0]             

In [21]:
hist = full_model.fit(x_train_features,
                 {'energy': y_train, 'decoder': x_train_features},
                 epochs=500,
                 batch_size=32,
                 shuffle=True,
                 validation_data=(x_test_features, {'energy': y_test, 'decoder': x_test_features}),
                 verbose = 1)

Epoch 1/500
596/596 [==============================] - 2s 4ms/step - loss: 0.9243 - decoder_loss: 0.6953 - energy_loss: 0.2290 - val_loss: 0.7525 - val_decoder_loss: 0.5896 - val_energy_loss: 0.1629
Epoch 2/500
596/596 [==============================] - 2s 4ms/step - loss: 0.7114 - decoder_loss: 0.5495 - energy_loss: 0.1619 - val_loss: 0.6662 - val_decoder_loss: 0.5154 - val_energy_loss: 0.1508
Epoch 3/500
596/596 [==============================] - 2s 4ms/step - loss: 0.6387 - decoder_loss: 0.4889 - energy_loss: 0.1498 - val_loss: 0.6049 - val_decoder_loss: 0.4606 - val_energy_loss: 0.1442
Epoch 4/500
596/596 [==============================] - 2s 4ms/step - loss: 0.5807 - decoder_loss: 0.4364 - energy_loss: 0.1443 - val_loss: 0.5706 - val_decoder_loss: 0.4136 - val_energy_loss: 0.1570
Epoch 5/500
596/596 [==============================] - 2s 4ms/step - loss: 0.5361 - decoder_loss: 0.3983 - energy_loss: 0.1377 - val_loss: 0.5247 - val_decoder_loss: 0.3847 - val_energy_loss: 0.1400
Epoch

596/596 [==============================] - 2s 4ms/step - loss: 0.3055 - decoder_loss: 0.2219 - energy_loss: 0.0836 - val_loss: 0.3435 - val_decoder_loss: 0.2195 - val_energy_loss: 0.1240
Epoch 43/500
596/596 [==============================] - 2s 4ms/step - loss: 0.3044 - decoder_loss: 0.2213 - energy_loss: 0.0831 - val_loss: 0.3430 - val_decoder_loss: 0.2206 - val_energy_loss: 0.1224
Epoch 44/500
596/596 [==============================] - 2s 4ms/step - loss: 0.3025 - decoder_loss: 0.2208 - energy_loss: 0.0818 - val_loss: 0.3328 - val_decoder_loss: 0.2203 - val_energy_loss: 0.1125
Epoch 45/500
596/596 [==============================] - 2s 4ms/step - loss: 0.3036 - decoder_loss: 0.2203 - energy_loss: 0.0832 - val_loss: 0.3445 - val_decoder_loss: 0.2210 - val_energy_loss: 0.1235
Epoch 46/500
596/596 [==============================] - 2s 4ms/step - loss: 0.3019 - decoder_loss: 0.2203 - energy_loss: 0.0816 - val_loss: 0.3388 - val_decoder_loss: 0.2192 - val_energy_loss: 0.1196
Epoch 47/500


596/596 [==============================] - 2s 4ms/step - loss: 0.2861 - decoder_loss: 0.2155 - energy_loss: 0.0706 - val_loss: 0.3425 - val_decoder_loss: 0.2153 - val_energy_loss: 0.1273
Epoch 84/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2844 - decoder_loss: 0.2140 - energy_loss: 0.0704 - val_loss: 0.3293 - val_decoder_loss: 0.2137 - val_energy_loss: 0.1156
Epoch 85/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2806 - decoder_loss: 0.2112 - energy_loss: 0.0694 - val_loss: 0.3247 - val_decoder_loss: 0.2085 - val_energy_loss: 0.1162
Epoch 86/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2776 - decoder_loss: 0.2071 - energy_loss: 0.0705 - val_loss: 0.3221 - val_decoder_loss: 0.2059 - val_energy_loss: 0.1162
Epoch 87/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2743 - decoder_loss: 0.2041 - energy_loss: 0.0702 - val_loss: 0.3198 - val_decoder_loss: 0.2041 - val_energy_loss: 0.1156
Epoch 88/500


Epoch 124/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2633 - decoder_loss: 0.1986 - energy_loss: 0.0647 - val_loss: 0.3187 - val_decoder_loss: 0.1984 - val_energy_loss: 0.1203
Epoch 125/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2626 - decoder_loss: 0.1985 - energy_loss: 0.0641 - val_loss: 0.3098 - val_decoder_loss: 0.1986 - val_energy_loss: 0.1111
Epoch 126/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2633 - decoder_loss: 0.1987 - energy_loss: 0.0646 - val_loss: 0.3117 - val_decoder_loss: 0.1992 - val_energy_loss: 0.1126
Epoch 127/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2632 - decoder_loss: 0.1986 - energy_loss: 0.0646 - val_loss: 0.3116 - val_decoder_loss: 0.1976 - val_energy_loss: 0.1140
Epoch 128/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2633 - decoder_loss: 0.1985 - energy_loss: 0.0648 - val_loss: 0.3178 - val_decoder_loss: 0.1987 - val_energy_loss: 0.

Epoch 165/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2587 - decoder_loss: 0.1981 - energy_loss: 0.0606 - val_loss: 0.3138 - val_decoder_loss: 0.1983 - val_energy_loss: 0.1155
Epoch 166/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2590 - decoder_loss: 0.1980 - energy_loss: 0.0610 - val_loss: 0.3148 - val_decoder_loss: 0.1985 - val_energy_loss: 0.1163
Epoch 167/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2589 - decoder_loss: 0.1981 - energy_loss: 0.0608 - val_loss: 0.3308 - val_decoder_loss: 0.1981 - val_energy_loss: 0.1327
Epoch 168/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2580 - decoder_loss: 0.1978 - energy_loss: 0.0602 - val_loss: 0.3174 - val_decoder_loss: 0.1983 - val_energy_loss: 0.1192
Epoch 169/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2586 - decoder_loss: 0.1981 - energy_loss: 0.0605 - val_loss: 0.3168 - val_decoder_loss: 0.1984 - val_energy_loss: 0.

...

Epoch 329/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2177 - decoder_loss: 0.1644 - energy_loss: 0.0534 - val_loss: 0.2886 - val_decoder_loss: 0.1636 - val_energy_loss: 0.1251
Epoch 330/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2178 - decoder_loss: 0.1643 - energy_loss: 0.0535 - val_loss: 0.2802 - val_decoder_loss: 0.1641 - val_energy_loss: 0.1162
Epoch 331/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2169 - decoder_loss: 0.1644 - energy_loss: 0.0525 - val_loss: 0.2848 - val_decoder_loss: 0.1639 - val_energy_loss: 0.1208
Epoch 332/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2179 - decoder_loss: 0.1651 - energy_loss: 0.0528 - val_loss: 0.2737 - val_decoder_loss: 0.1628 - val_energy_loss: 0.1110
Epoch 333/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2183 - decoder_loss: 0.1644 - energy_loss: 0.0539 - val_loss: 0.2772 - val_decoder_loss: 0.1631 - val_energy_loss: 0.

Epoch 370/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2167 - decoder_loss: 0.1643 - energy_loss: 0.0524 - val_loss: 0.2778 - val_decoder_loss: 0.1637 - val_energy_loss: 0.1141
Epoch 371/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2171 - decoder_loss: 0.1644 - energy_loss: 0.0527 - val_loss: 0.2811 - val_decoder_loss: 0.1648 - val_energy_loss: 0.1163
Epoch 372/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2160 - decoder_loss: 0.1644 - energy_loss: 0.0516 - val_loss: 0.2821 - val_decoder_loss: 0.1646 - val_energy_loss: 0.1174
Epoch 373/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2167 - decoder_loss: 0.1642 - energy_loss: 0.0525 - val_loss: 0.2813 - val_decoder_loss: 0.1643 - val_energy_loss: 0.1170
Epoch 374/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2166 - decoder_loss: 0.1642 - energy_loss: 0.0524 - val_loss: 0.2812 - val_decoder_loss: 0.1644 - val_energy_loss: 0.

Epoch 411/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2154 - decoder_loss: 0.1643 - energy_loss: 0.0511 - val_loss: 0.2957 - val_decoder_loss: 0.1664 - val_energy_loss: 0.1293
Epoch 412/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2150 - decoder_loss: 0.1642 - energy_loss: 0.0508 - val_loss: 0.2768 - val_decoder_loss: 0.1631 - val_energy_loss: 0.1138
Epoch 413/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2149 - decoder_loss: 0.1642 - energy_loss: 0.0507 - val_loss: 0.2913 - val_decoder_loss: 0.1644 - val_energy_loss: 0.1269
Epoch 414/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2151 - decoder_loss: 0.1640 - energy_loss: 0.0510 - val_loss: 0.2804 - val_decoder_loss: 0.1640 - val_energy_loss: 0.1163
Epoch 415/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2159 - decoder_loss: 0.1643 - energy_loss: 0.0516 - val_loss: 0.2738 - val_decoder_loss: 0.1635 - val_energy_loss: 0.

Epoch 452/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2142 - decoder_loss: 0.1642 - energy_loss: 0.0500 - val_loss: 0.2807 - val_decoder_loss: 0.1634 - val_energy_loss: 0.1174
Epoch 453/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2141 - decoder_loss: 0.1641 - energy_loss: 0.0500 - val_loss: 0.2786 - val_decoder_loss: 0.1640 - val_energy_loss: 0.1146
Epoch 454/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2143 - decoder_loss: 0.1644 - energy_loss: 0.0499 - val_loss: 0.2837 - val_decoder_loss: 0.1643 - val_energy_loss: 0.1194
Epoch 455/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2140 - decoder_loss: 0.1641 - energy_loss: 0.0499 - val_loss: 0.2798 - val_decoder_loss: 0.1639 - val_energy_loss: 0.1159
Epoch 456/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2148 - decoder_loss: 0.1642 - energy_loss: 0.0506 - val_loss: 0.2749 - val_decoder_loss: 0.1634 - val_energy_loss: 0.

Epoch 493/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2134 - decoder_loss: 0.1642 - energy_loss: 0.0492 - val_loss: 0.2765 - val_decoder_loss: 0.1628 - val_energy_loss: 0.1137
Epoch 494/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2133 - decoder_loss: 0.1640 - energy_loss: 0.0493 - val_loss: 0.2851 - val_decoder_loss: 0.1639 - val_energy_loss: 0.1212
Epoch 495/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2133 - decoder_loss: 0.1640 - energy_loss: 0.0493 - val_loss: 0.2887 - val_decoder_loss: 0.1645 - val_energy_loss: 0.1241
Epoch 496/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2140 - decoder_loss: 0.1643 - energy_loss: 0.0498 - val_loss: 0.2892 - val_decoder_loss: 0.1638 - val_energy_loss: 0.1254
Epoch 497/500
596/596 [==============================] - 2s 4ms/step - loss: 0.2131 - decoder_loss: 0.1640 - energy_loss: 0.0491 - val_loss: 0.2797 - val_decoder_loss: 0.1637 - val_energy_loss: 0.

In [127]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Features (InputLayer)        [(None, 62)]              0         
_________________________________________________________________
dense_40 (Dense)             (None, 62)                3906      
_________________________________________________________________
dense_41 (Dense)             (None, 32)                2016      
_________________________________________________________________
dense_42 (Dense)             (None, 16)                528       
Total params: 6,450
Trainable params: 6,450
Non-trainable params: 0
_________________________________________________________________


In [128]:
decoder = decoder()

In [129]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense_43 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_44 (Dense)             (None, 62)                2046      
Total params: 2,590
Trainable params: 2,590
Non-trainable params: 0
_________________________________________________________________


In [136]:
create_model()

ValueError: The first argument to `Layer.call` must always be passed.

In [11]:
latent_dim = 16
def model():
    #encoder
    features = Input(shape=(62,),name='Features')
    X = Dense(62, activation = 'relu')(features)
    X1 = Dense(32, activation = 'relu')(X)
    encoded = Dense(latent_dim, activation = 'relu', name = 'encoder')(X1)
    
    #difference model
    X2 = Dense(32,activation='relu')(encoded)
    X3 = Dropout(0.2)(X2)
    X4 = Dense(6,activation='relu')(X3)
    output = Dense(1, activation='linear', name = 'energy')(X4)
    
    #decoder 
    X5 = Dense(32, activation = 'relu')(encoded)
    decoded = Dense(62, activation = 'linear', name = 'autoencoder')(X5)
    
    model = Model(inputs = features, outputs = [output, decoded])
    return model
    
    

In [12]:
model = model()

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Features (InputLayer)           [(None, 62)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 62)           3906        Features[0][0]                   
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           2016        dense[0][0]                      
__________________________________________________________________________________________________
encoder (Dense)                 (None, 16)           528         dense_1[0][0]                    
______________________________________________________________________________________________

In [14]:
model.compile(loss={'energy': 'mse', 
                    'autoencoder': 'mse'},
              optimizer='adam'
)

In [71]:
hist = model.fit(x_train_features,
                 {'energy': y_train, 'autoencoder': x_train_features},
                 epochs=500,
                 batch_size=32,
                 shuffle=True,
                 validation_data=(x_test_features, {'energy': y_test, 'autoencoder': x_test_features}),
                 verbose = 1)

Epoch 1/500


ValueError: in user code:

    C:\Users\keith\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\keith\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\keith\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\keith\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\keith\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=s

In [19]:
tf.test.is_gpu_available

<function tensorflow.python.framework.test_util.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)>